In [35]:
import numpy as np 

class PDMA:
    """Pentadiagonal matrix solver
    Parameters
    ----------
        mat : SparseMatrix
            Pentadiagonal matrix with diagonals in offsets
            -4, -2, 0, 2, 4
        neumann : bool, optional
            Whether matrix represents a Neumann problem, where
            the first index is known as the mean value and we
            solve for slice(1, N-3).
            If `mat` is a :class:`.SpectralMatrix`, then the
            `neumann` keyword is ignored and the information
            extracted from the matrix.
    """

    def __init__(self, mat, neumann=False):
        self.mat = mat
        self.N = 0
        self.d0 = np.zeros(0)
        self.d1 = None
        self.d2 = None
        self.A = None
        self.L = None
        self.neumann = neumann

    def init(self):
        """Initialize and allocate solver"""
        B = self.mat
        shape = self.mat.shape[1]
        # Broadcast in case diagonal is simply a constant.
        self.d0 = np.broadcast_to(np.diag(B,0), shape).copy()#*B.scale
        self.d1 = np.broadcast_to(np.diag(B,2), shape-2).copy()#*B.scale
        self.d2 = np.broadcast_to(np.diag(B,4), shape-4).copy()#*B.scale
        if self.neumann:
            self.d0[0] = 1
            self.d1[0] = 0
            self.d2[0] = 0
        self.l1 = np.broadcast_to(np.diag(B,-2), shape-2).copy()#*B.scale
        self.l2 = np.broadcast_to(np.diag(B,-4), shape-4).copy()#*B.scale
        self.PDMA_LU(self.l2, self.l1, self.d0, self.d1, self.d2)
            
    @staticmethod
    def PDMA_LU(a, b, d, e, f): # pragma: no cover
        """LU decomposition"""
        n = d.shape[0]
        m = e.shape[0]
        k = n - m

        for i in range(n-2*k):
            lam = b[i]/d[i]
            d[i+k] -= lam*e[i]
            e[i+k] -= lam*f[i]
            b[i] = lam
            lam = a[i]/d[i]
            b[i+k] -= lam*e[i]
            d[i+2*k] -= lam*f[i]
            a[i] = lam

        i = n-4
        lam = b[i]/d[i]
        d[i+k] -= lam*e[i]
        b[i] = lam
        i = n-3
        lam = b[i]/d[i]
        d[i+k] -= lam*e[i]
        b[i] = lam


    @staticmethod
    def PDMA_Solve(a, b, d, e, f, h, axis=0): # pragma: no cover
        """Symmetric solve (for testing only)"""
        n = d.shape[0]
        bc = h

        bc[2] -= b[0]*bc[0]
        bc[3] -= b[1]*bc[1]
        for k in range(4, n):
            bc[k] -= (b[k-2]*bc[k-2] + a[k-4]*bc[k-4])

        bc[n-1] /= d[n-1]
        bc[n-2] /= d[n-2]
        bc[n-3] = (bc[n-3]-e[n-3]*bc[n-1])/d[n-3]
        bc[n-4] = (bc[n-4]-e[n-4]*bc[n-2])/d[n-4]
        for k in range(n-5, -1, -1):
            bc[k] = (bc[k]-e[k]*bc[k+2]-f[k]*bc[k+4])/d[k]
            
    def solve(self,x):
        self.PDMA_Solve(self.l2, self.l1, self.d0, self.d1, self.d2, x, axis=0)

In [39]:
N = 10
l2 = np.random.random(N-4)
l1 = np.random.random(N-2)
d0 = np.random.random(N+0)
u1 = np.random.random(N-2)
u2 = np.random.random(N-4)
x = np.random.random(N)

A = np.diag(l2,-4)+np.diag(l1,-2)+np.diag(d0,0)+np.diag(u1,+2)+np.diag(u2,+4)
np.linalg.solve(A,x)

array([-0.36702836,  0.00951234,  1.86959482,  1.49593577, -0.90858678,
       -1.1843992 ,  0.07224869,  1.12629228,  1.2845515 ,  0.90442261])

In [40]:
P = PDMA(A)
P.init()
print(x)
P.solve(x)
print(x)

[0.77809456 0.02890255 0.7192478  0.72930258 0.32962444 0.84240715
 0.71712376 0.81441328 0.01630718 0.93305048]
[-0.36702836  0.00951234  1.86959482  1.49593577 -0.90858678 -1.1843992
  0.07224869  1.12629228  1.2845515   0.90442261]


In [21]:
P.mat[0]

array([0.67909353, 0.        , 0.01144835, 0.        , 0.72930119,
       0.        , 0.        , 0.        , 0.        , 0.        ])